<a href="https://colab.research.google.com/github/huyhoang1996vn/my-ml/blob/master/TFIDF_vectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [1]:
!pip install ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=e51bc179d7bfb8999881d53efa7c8975fe925944837d69f7405242127d229dca
  Stored in directory: /root/.cache/pip/wheels/5f/d4/d7/4189b07b5902ee9f3ce0dbb14909fbe8037c39d6c63ffd49c9
Successfully built htmlmin
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
     

To read an .ods file into a pandas DataFrame, you can use the `read_excel` function and specify the engine as `odf`.

In [2]:
!pip install odfpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160673 sha256=6a4c3569a7c044db1f38ddbf3dbdb58977f110cca676986a7ce54246e6a72caf
  Stored in directory: /root/.cache/pip/wheels/36/5d/63/8243a7ee78fff0f944d638fd0e66d7278888f5e2285d7346b6
Successfully built odfpy


In [18]:
# Read the .ods file into a pandas DataFrame
df = pd.read_excel('final_project.ods', engine='odf', dtype=str)

# Display the first 5 rows of the DataFrame
display(df.head())

,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,"Houston, TX","Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,"Seattle, WA","Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,"Dallas, TX",We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),"Austin, TX",Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


In [5]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Profiling Report", explorative=True)
profile.to_file("profiling_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 6/6 [00:08<00:00,  1.34s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
df.isna().sum()

,0
title,0
location,0
description,1
function,0
industry,0
career_level,0


In [29]:
def extract_second_part_of_location(location):
  """
  Checks if a location string contains a comma, splits it if so, and returns the second part.

  Args:
    location: The location string.

  Returns:
    The second part of the location string if a comma is found and splitting is successful,
    otherwise returns the original location.
  """
  if isinstance(location, str) and ',' in location:
    parts = location.split(',', 1) # Split only on the first comma
    if len(parts) > 1:
      return parts[1].strip() # Return the second part and remove leading/trailing whitespace
  return location

# Apply the function to the 'location' column
df['location'] = df['location'].apply(extract_second_part_of_location)

# Display the updated DataFrame
display(df.head())

,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,TX,"Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,WA,"Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,TX,We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),TX,Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


In [30]:
df.dropna(subset=['description'], inplace=True)
display(df.isna().sum())
display(df.head())
display(df.describe())

,0
title,0
location,0
description,0
function,0
industry,0
career_level,0


,title,location,description,function,industry,career_level
0,Technical Professional Lead - Process,TX,"Responsible for the study, design, and specifi...",production_manufacturing,Machinery and Industrial Facilities Engineering,senior_specialist_or_project_manager
1,Cnslt - Systems Eng- Midrange 1,WA,"Participates in design, development and implem...",information_technology_telecommunications,Financial Services,senior_specialist_or_project_manager
2,SharePoint Developers and Solution Architects,TX,We are currently in need of Developers who can...,consulting,IT Consulting,senior_specialist_or_project_manager
3,Business Information Services - Strategic Acco...,North Carolina,Experian is seeking an experienced Account Exe...,sales,"Security, Risk, Restructuring Consulting",senior_specialist_or_project_manager
4,Strategic Development Director (procurement),TX,Â Want to join a world-class global procuremen...,procurement_materials_logistics,Information Technology,bereichsleiter


,title,location,description,function,industry,career_level
count,8073,8073,8073,8073,8073,8073
unique,6789,97,7973,19,352,6
top,Account Manager,CA,Practice Medicine in Northern California with ...,sales,Software Companies,senior_specialist_or_project_manager
freq,58,1732,5,3150,688,4337


In [31]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('career_level', axis=1), df['career_level'], test_size=0.2, random_state=42, stratify=df['career_level'])

In [8]:
y_train.value_counts()

,count
career_level,
senior_specialist_or_project_manager,3469
manager_team_leader,2138
bereichsleiter,768
director_business_unit_leader,56
specialist,24
managing_director_small_medium_company,3


In [24]:
y_test.value_counts()

,count
career_level,
senior_specialist_or_project_manager,868
manager_team_leader,534
bereichsleiter,192
director_business_unit_leader,14
specialist,6
managing_director_small_medium_company,1


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a corpus of documents
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?'
]

# Create an instance of TfidfVectorizer
# Optional parameters can be added, such as stop_words='english', ngram_range=(1, 2), etc.
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the corpus and transform the text data into TF-IDF features
X = vectorizer.fit_transform(corpus)

# Get the feature names (words/tokens) learned by the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Convert the sparse matrix to a dense array for inspection (optional)
tfidf_matrix_array = X.toarray()

# Print the feature names and the TF-IDF matrix
print("Feature Names:", feature_names)
print(len(tfidf_matrix_array))
print(len(tfidf_matrix_array[0]))
print("TF-IDF Matrix:\n", tfidf_matrix_array[0])

Feature Names: ['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
4
9
TF-IDF Matrix:
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
 0.38408524 0.         0.38408524]


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Convert text columns to string type to handle potential non-string data
# x_train['title'] = x_train['title'].astype(str)
# x_train['industry'] = x_train['industry'].astype(str)
# x_train['function'] = x_train['function'].astype(str)
# x_train['description'] = x_train['description'].astype(str)
# x_train['location'] = x_train['location'].astype(str)

# Create separate TfidfVectorizer instances for each text column
tfidf_title = TfidfVectorizer()
tfidf_industry = TfidfVectorizer()
tfidf_function = TfidfVectorizer()
tfidf_description = TfidfVectorizer(ngram_range=(1, 2)) # Using ngram_range=(1,2) for description as before

# Create a OneHotEncoder instance for the location column
one_hot_location = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf_title', tfidf_title, 'title'),
        ('tfidf_industry', tfidf_industry, 'industry'),
        ('tfidf_function', tfidf_function, 'function'),
        ('tfidf_description', tfidf_description, 'description'),
        ('one_hot_location', one_hot_location, ['location']),
    ])

# Fit and transform the training data
x_train_processed = preprocessor.fit_transform(x_train)

# Display the shape of the processed training data
# print(x_train_processed.shape)

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

reg=Pipeline(steps=[('preprocessor',preprocessor),('regressor', RandomForestClassifier())])
reg.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('tfidf_title',
                                                  TfidfVectorizer(), 'title'),
                                                 ('tfidf_industry',
                                                  TfidfVectorizer(),
                                                  'industry'),
                                                 ('tfidf_function',
                                                  TfidfVectorizer(),
                                                  'function'),
                                                 ('tfidf_description',
                                                  TfidfVectorizer(ngram_range=(1,
                                                                               2)),
                                                  'description'),
                                                 ('one_hot_location',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['location'])])),
                ('regressor', RandomForestClassifier())])

In [39]:
y_predict = reg.predict(x_test)

In [40]:
from sklearn.metrics import classification_report
report=classification_report(y_test, y_predict)
print(report)

                                        precision    recall  f1-score   support

                        bereichsleiter       0.67      0.05      0.10       192
         director_business_unit_leader       1.00      0.29      0.44        14
                   manager_team_leader       0.62      0.55      0.58       534
managing_director_small_medium_company       0.00      0.00      0.00         1
  senior_specialist_or_project_manager       0.73      0.94      0.82       868
                            specialist       1.00      0.17      0.29         6

                              accuracy                           0.70      1615
                             macro avg       0.67      0.33      0.37      1615
                          weighted avg       0.69      0.70      0.65      1615



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_predict)
# precision = precision_score(y_test, y_predict, pos_label='yes')
# recall = recall_score(y_test, y_predict, pos_label='yes')
# f1 = f1_score(y_test, y_predict, pos_label='yes')

print(f"Accuracy: {accuracy:.2f}")
# print(f"Precision: {precision:.2f}")
# print(f"Recall: {recall:.2f}")
# print(f"F1-score: {f1:.2f}")

Accuracy: 0.70
